In [2]:
import numpy as np
import regex as re

In [3]:
import re

def separate_string(string):
  # Initialize variables to track the start and end indices of speech and paragraphs
  speech_start = None
  speech_end = None
  paragraph_start = 0
  paragraph_end = 0

  # Initialize list to store the separated strings
  separated_strings = []

  # Iterate through the string character by character
  for i, c in enumerate(string):
    # Check if the character is the start or end of speech
    if c == '"':
      if speech_start is None:
        speech_start = i
      else:
        speech_end = i
        # Extract the speech and add it to the list
        separated_strings.append(string[speech_start:speech_end + 1])
        speech_start = None
        speech_end = None
    # Check if the character is the start or end of a paragraph
    elif c == '<':
      # Extract the paragraph and add it to the list
      paragraph = string[paragraph_start:paragraph_end]
      separated_strings.append(paragraph)
      paragraph_start = i
      paragraph_end = i
      # Check if the tag is a paragraph tag
      if re.match(r'<p[^>]*>', string[i:]):
        paragraph_end = i + re.search(r'</p>', string[i:]).start() + 4
  # Add the final paragraph to the list (if there is one)
  if paragraph_end > paragraph_start:
    paragraph = string[paragraph_start:paragraph_end]
    separated_strings.append(paragraph)
  # Replace multiple consecutive spaces with a single space
  separated_strings = [re.sub(r' +', ' ', s) for s in separated_strings]
# Replace newline characters within sentences with a single space
  separated_strings = [s.replace('\n', ' ') for s in separated_strings]
    
# Remove the <p> tags from the paragraphs
  separated_strings = [re.sub(r'<p[^>]*>', '', s) for s in separated_strings]
  separated_strings = [re.sub(r'</p>', '', s) for s in separated_strings]
    
  # Remove empty strings from the list
  separated_strings = [s for s in separated_strings if s]
    
  return separated_strings


In [4]:
import re

def extract_p_tag_info(p_tag):
    # Initialize a dictionary to store the attributes and their values
    attributes = {}

    # Find the attributes and their values using a regular expression
    for match in re.finditer(r'(\S+)="(.*?)"', p_tag):
        attribute, value = match.groups()
        attributes[attribute] = value

    # Find the text inside the <p> tag using another regular expression
    text = re.search(r'>([^<]*)<', p_tag).group(1)

    # Replace multiple consecutive spaces with a single space
    text = re.sub(r' +', ' ', text)

    # Replace newline characters within sentences with a single space
    text = text.replace('\n', ' ')

    # Remove empty strings from the list
    text = text.strip()

    # Return the attributes as a dictionary and the text as a string
    return [attributes, text]



In [5]:
import re

def extract_div_tag_info(div_tag):
  # Initialize a dictionary to store the attributes and their values
  attributes = {}

  # Find the attributes and their values using a regular expression
  for match in re.finditer(r'(\S+)=["\']?((?:.(?!["\']?\s+(?:\S+)=|[>"\']))+.)["\']?', div_tag):
    attribute, value = match.groups()
    attributes[attribute] = value

  # Find the text inside the <p> tag using another regular expression
  text = re.search(r'>([^<]*)<', div_tag).group(1)

  # Replace multiple consecutive spaces with a single space
  text = re.sub(r' +', ' ', text)

  # Replace newline characters within sentences with a single space
  text = text.replace('\n', ' ')

  # Remove empty strings from the list
  text = text.strip()

  # Return the attributes, values, and text as a list
  return [attributes, text]


In [6]:
import re

def separate_string(string):
  # Initialize variables to track the start and end indices of paragraphs
  paragraph_start = 0
  paragraph_end = 0

  # Initialize list to store the separated strings
  separated_strings = []

  # Iterate through the string character by character
  for i, c in enumerate(string):
    # Check if the character is the start or end of a paragraph
    if c == '<':
      # Extract the paragraph and add it to the list
      paragraph = string[paragraph_start:paragraph_end]
      separated_strings.append(paragraph)
      paragraph_start = i
      paragraph_end = i
      # Check if the tag is a paragraph tag
      if re.match(r'<p[^>]*>', string[i:]) and not re.match(r'<progress', string[i:]):
        paragraph_end = i + re.search(r'</p>', string[i:]).start() + 4
  # Add the final paragraph to the list (if there is one)
  if paragraph_end > paragraph_start:
    paragraph = string[paragraph_start:paragraph_end]
    separated_strings.append(paragraph)

  # Replace newline characters within sentences with spaces
  separated_strings = [s.replace('\n', ' ') for s in separated_strings]

  # Remove empty strings
  separated_strings = [s for s in separated_strings if s]

  # Iterate through the separated strings
  for i, s in enumerate(separated_strings):
    # Check if the string is a paragraph
    if s.startswith('<p') and not s.startswith('<progress'):
      # Extract the attributes and text inside the <p> tag using the extract_p_tag_info() function
      p_tag_info = extract_p_tag_info(s)
      # Replace the <p> tag with the extracted information
      separated_strings[i] = p_tag_info
    if s.startswith('<div'):
      div_tag_info = extract_div_tag_info(s)
      separated_strings[i] = div_tag_info
 
  # Return the separated strings
  return separated_strings


In [7]:
import os
import json

# Set the path to the folder containing the HTML files
folder_path = "chapters"

# Create an empty dictionary to store the file contents
html_dict = {}

# Create a new folder called "json_chapters" if it doesn't already exist
if not os.path.exists("json_chapters"):
    os.makedirs("json_chapters")

# Loop through all files in the folder with a .html extension
for filename in os.listdir(folder_path):
    if filename.endswith(".html"):
        # Set the key in the dictionary as the filename without the .html extension
        key = os.path.splitext(filename)[0].replace(" ", "")
        
        # Set the path to the HTML file
        file_path = os.path.join(folder_path, filename)
        
        # Open the HTML file and read its contents as a string
        with open(file_path, "r") as f:
            html_string = f.read()
            
        # Add the file contents to the dictionary
        html_dict[key] = html_string

# Save each entry in the dictionary as a JSON file
for key, value in html_dict.items():
    # Set the filename for the JSON file
    json_filename = f"{key}.json"
    
    # Create a dictionary with the file contents
    data_dict = {"content": separate_string(value)}
    
    # Convert the dictionary to a JSON string
    json_string = json.dumps(data_dict)
    
    # Set the path to the JSON file
    json_file_path = os.path.join("json_chapters", json_filename)
    
    # Save the JSON string to a file in the "json_chapters" folder
    with open(json_file_path, "w") as outfile:
        outfile.write(json_string)
        
# Print a message to confirm that all files were saved successfully
print("JSON files saved successfully!")


JSON files saved successfully!
